In [ ]:
# An example notebook to download and visualise boundary data.

In [ ]:
# Basics
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# Web requests
import json
import requests

In [ ]:
# Maps and geodata
import collections
import geopandas as gpd
import matplotlib.patheffects
import matplotlib_scalebar.scalebar
import textwrap

Data
==

Boundaries
--

In [ ]:
# Ward boundaries for Lewisham (May 2023), generalised to 20m
# Source: https://geoportal.statistics.gov.uk/datasets/ons::wards-may-2023-boundaries-uk-bgc/api
# 
# With API export options:
# - Where LD23NM = 'Lewisham'
# - Output Spatial Reference = 27700 (OSGB36 / British National Grid)

# Fetch
response = requests.get("https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/WD_MAY_2023_UK_BGC/FeatureServer/0/query?where=LAD23NM%20%3D%20'LEWISHAM'&outFields=*&outSR=27700&f=json")
data = json.loads(response.content)

# Save as JSON
with open('data/downloaded/Lewisham_Wards_May_2023_Boundaries_UK_BGC.json', 'w') as f:
    json.dump(data, f)

print("Downloaded {:,.1f} kB".format(len(response.content) / 1024))

In [ ]:
# Load in GeoPandas
wd23 = gpd.read_file('data/downloaded/Lewisham_Wards_May_2023_Boundaries_UK_BGC.json')
print(f"Loaded boundaries for {len(wd23)} wards")

In [ ]:
wd23.plot()

Population estimates
--

In [ ]:
# 2021 Census: ward-level population estimates
# Source: https://data.london.gov.uk/census/2021-ward-and-lsoa-estimates/

# Fetch
response = requests.get("https://data.london.gov.uk/download/2021-census-wards-demography-and-migration/b4fca9c8-30fa-4e66-858c-cbeb623fda9c/Usual%20Residents.xlsx")

# Save as XLSX
with open('data/downloaded/London_2021_Census_by_Ward-Usual_Residents.xlsx', 'wb') as f:
    f.write(response.content)
    
print("Downloaded {:,.1f} kB".format(len(response.content) / 1024))

In [ ]:
# Load in Pandas
pop = pd.read_excel('data/downloaded/London_2021_Census_by_Ward-Usual_Residents.xlsx',
                    sheet_name='2021')
pop.sample(2)

Maps & charts
==

Tools
--

In [ ]:
# Label helpers for maps

# Manual label location adjustments to improve legibility.
# Offsets are (x, y) tuples with national grid units.
# Default offset is (0, 0).
map_ward_label_offsets = collections.defaultdict(lambda: (0, 0), {
    'Bellingham': (-200, 150),
    'Brockley': (0, 40),
    'Catford South': (-100, 0),
    'Downham': (-200, -200),
    'Lewisham Central': (0, -100),
    'Sydenham': (0, -100),
    'Telegraph Hill': (0, -100),
})

def wrap_label(text, length):
    return "\n".join(textwrap.wrap(text, length))

def adjust_label_coords(ward_name, xy):
    xy_offset = map_ward_label_offsets[ward_name]
    return (xy[0] + xy_offset[0], xy[1] + xy_offset[1])

def plot_label(text, xy, size=12, color='black', ha='center', stroke_width=2, stroke_color='white'):
    plt.gca().annotate(text, xy=xy, ha=ha, 
                       size=size, c=color,
                       path_effects=[
                           matplotlib.patheffects.withStroke(
                               linewidth=stroke_width,
                               foreground=stroke_color)
                       ])

Ward map
--

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7.5, 9), facecolor='white', dpi=150)

plt.title(f"Lewisham Wards (May 2023)", fontsize=14)

# Borders
wd23.plot(ax=ax, linewidth=1, facecolor='#88ccee', edgecolor='white')

# Labels
wd23.apply(lambda x: plot_label(wrap_label(x['WD23NM'], 15),
                                adjust_label_coords(x['WD23NM'], x.geometry.centroid.coords[0]),
                                size=11, color='black'), 
           axis=1)

# Scale bar
ax.add_artist(matplotlib_scalebar.scalebar.ScaleBar(1,
    location='lower right',
    fixed_value=1, fixed_units='km', 
    font_properties={'size': 10},
    color='#88ccee', box_alpha=0, scale_loc='top',
))

# Remove decoration
plt.box(False)
plt.xticks([])
plt.yticks([])

fig.tight_layout()

# Export
plt.savefig("data/processed/Lewisham_Wards_May_2023.png", dpi=150, bbox_inches='tight')

Population distribution
--

In [ ]:
# Lewisham ward populations
lbl_pop = pop[pop['ward code'].isin(wd23.WD23CD)].\
    sort_values(by='All usual residents', ascending=True).\
    reset_index(drop=True) # We will use the index for vertical placement

# Export
lbl_pop.to_csv('data/processed/Lewisham_Ward_Population_2021_Census.csv', index=True)

lbl_pop.head(2)

In [ ]:
# Display as a formatted table
lbl_pop[['ward name', 'All usual residents']].\
    iloc[::-1].\
    rename(columns={'ward name': 'Ward'}).\
    set_index('Ward').\
    style.format('{:,}')

In [ ]:
# Display as a bar chart
fig, ax = plt.subplots(1, 1, figsize=(5, 6), facecolor='white', dpi=150)

with sns.axes_style('ticks'):
    
    plt.title('Lewisham Population by Ward (2021 Census)')
    
    p = plt.barh(y=lbl_pop.index,
                 width=lbl_pop['All usual residents'] / 1000,
                 height=0.8,
                 edgecolor='white', linewidth=0.5)
    
    plt.bar_label(p, lbl_pop['All usual residents'].astype(int).map(lambda v: '{:,.1f}'.format(v/1000)),
                  padding=5, color='darkgray')

    plt.xlabel('Usual resident population (thousands)')
    ax.get_xaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
    plt.yticks(lbl_pop.index, lbl_pop['ward name'])

    plt.figtext(0.5, 0, 'Source: ONS 2023 best-fit estimates', 
                va='top', ha='center', fontsize=9, color='#888888')

    sns.despine()
    fig.tight_layout()
    
# Export
plt.savefig("data/processed/Lewisham_Ward_Population_2021_Census.png", dpi=150, bbox_inches='tight')

Choropleth map
--

In [ ]:
# Join boundaries with indicator data
d = wd23.set_index('WD23CD').join(pop.set_index('ward code'))
d.sample(2)

In [ ]:
# Display as choropleth map
fig, ax = plt.subplots(1, 1, figsize=(7.5, 9), facecolor='white', dpi=150)

plt.title("Lewisham Population by Ward (2021 Census)", fontsize=14)

# Palette
min_cmap_idx = 0.4 # don't start with white
blues = matplotlib.colors.ListedColormap(matplotlib.colormaps['Blues'](np.linspace(min_cmap_idx, 1, 256)))

# Data
d.plot(ax=ax, column='All usual residents',
       legend=True,
       cmap=blues,
       
       # Option 1: no segmentation
       legend_kwds={
           'format': '%.0f',
           'shrink': 0.3,
       },
       
#        # Option 2: with segmentation
#        # mapclassify parameters for breaks
#        # See https://pysal.org/mapclassify/api.html
#        scheme='NaturalBreaks',
#        classification_kwds={'k': 4},
#        legend_kwds={
#            'title': 'Usual resident population',
#            # 'labels': [...] # Manual labels
#        },
)

# Borders
wd23.plot(ax=ax, linewidth=1, facecolor='none', edgecolor='white')

# Labels
d.apply(lambda x: plot_label('{:,}'.format(x['All usual residents']),
                                adjust_label_coords(x['WD23NM'], x.geometry.centroid.coords[0]),
                                size=9, color='black'), 
           axis=1)

# Scale bar
ax.add_artist(matplotlib_scalebar.scalebar.ScaleBar(1,
    location='lower right',
    fixed_value=1, fixed_units='km', 
    font_properties={'size': 10},
    color='#666666', box_alpha=0, scale_loc='top',
))

# Credits
plt.figtext(0.4, 0.15, 'Source: ONS 2023 best-fit estimates', 
            va='top', ha='center', fontsize=10, color='#888888')

# Remove decoration
plt.box(False)
plt.xticks([])
plt.yticks([])

fig.tight_layout()

# Export
plt.savefig("data/processed/Lewisham_Ward_Population_2021_Census_Map.png", dpi=150, bbox_inches='tight')